# Scikit-HEP ecosystem updates

**Note:** although I'm presenting in Jupyter, this is a talk, rather than a tutorial. You don't have to follow along.

<br><br><br><br><br>

## State of the ecosystem

<table width="100%"><tr style="background: white">
    <td align="left" width="50%"><img src="img/shells-border.png" width="95%"></td>
    <td align="right" width="50%"><img src="img/shells-hep.png" width="95%"></td>
</tr></table>

<br><br><br><br><br>

<img src="img/pip-allos-scikithep-log.png" width="100%">

<br><br><br><br><br>

## There are more tools than I could reasonably tell you about

And that's good:

   * approaching the ideal of each does one thing well
   * and is maintained by enthusiastic developers with recognition for their work
   * but we also know about each other and ensure that these tools work together

<br><br><br><br><br>

## Illustrative vertical slice: Awkward Array + Vector + fastjet (+ Uproot + hist)

Why these three (five)? I happen to know more about them.

<br><br>

<img src="img/awkward-logo.png" width="200px">

<p style="font-size: 14pt">Arrays of arbitrary data structures.</p>

<br><br>

<img src="img/vector-logo.png" width="200px">

<p style="font-size: 14pt">Arrays of 2D, 3D, and Lorentz vectors.</p>

<br><br>

<img src="img/fastjet-logo-300px.png" width="200px">

<p style="font-size: 14pt">Jet-finding on arrays of Lorentz vectors.</p>

<br><br>

<img src="img/uproot-logo.png" width="200px">

<p style="font-size: 14pt">Reading data from ROOT as arrays.</p>

<br><br>

<img src="img/hist-logo.png" width="200px">

<p style="font-size: 14pt">Filling, manipulating, and plotting histograms.</p>